In [1]:
import src.config as config

from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f'{current_date=}')

fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2024-01-27 05:00:00')


In [5]:
print(f'{fetch_data_to}')
print(f'{fetch_data_from} 28 days ago')

2024-01-27 05:00:00
2023-12-30 05:00:00 28 days ago


In [6]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:

    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)

    rides=load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [7]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

File 2022-12 was already in local storage
File 2023-01 was already in local storage


In [8]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 257/257 [00:00<00:00, 258.14it/s]


In [9]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2023-12-30 05:00:00,1,4
1,2023-12-30 06:00:00,0,4
2,2023-12-30 07:00:00,0,4
3,2023-12-30 08:00:00,0,4
4,2023-12-30 09:00:00,0,4
...,...,...,...
172699,2024-01-27 00:00:00,0,46
172700,2024-01-27 01:00:00,0,46
172701,2024-01-27 02:00:00,0,46
172702,2024-01-27 03:00:00,0,46


In [10]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()

feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Times-series data at hourly frequency",
    primary_key = ['pickup_location_id','pickup_hour'],
    event_time = 'pickup_hour',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
feature_group.insert(ts_data, write_options={"wait_for_jobs": False})

Uploading Dataframe: 0.00% |          | Rows 0/172704 | Elapsed Time: 00:00 | Remaining Time: ?